In [1]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import datetime


# This code is derived from AWS SageMaker Samples:
# https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/deepar_electricity
# https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/deepar_synthetic

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [3]:
# Set a good base job name when building different models
# It will help in identifying trained models and endpoints
with_categories = False
if with_categories:
    base_job_name = 'deepar-biketrain-with-categories'
else:
    base_job_name = 'deepar-biketrain-no-categories'

In [4]:
# Specify your bucket name
bucket = 'chandra-ml-sagemaker'
prefix = 'deepar/bikerental'

# This structure allows multiple training and test files for model development and testing
if with_categories:
    s3_data_path = "{}/{}/data_with_categories".format(bucket, prefix)
else:
    s3_data_path = "{}/{}/data".format(bucket, prefix)
    

s3_output_path = "{}/{}/output".format(bucket, prefix)

In [5]:
s3_data_path,s3_output_path

('chandra-ml-sagemaker/deepar/bikerental/data',
 'chandra-ml-sagemaker/deepar/bikerental/output')

In [6]:
# File name is referred as key name in S3
# Files stored in S3 are automatically replicated across
# three different availability zones in the region where the bucket was created.
# http://boto3.readthedocs.io/en/latest/guide/s3.html
def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [7]:
# Upload one or more training files and test files to S3
if with_categories:
    write_to_s3('train_with_categories.json',bucket,'deepar/bikerental/data_with_categories/train/train_with_categories.json')
    write_to_s3('test_with_categories.json',bucket,'deepar/bikerental/data_with_categories/test/test_with_categories.json')
else:
    write_to_s3('train.json',bucket,'deepar/bikerental/data/train/train.json')
    write_to_s3('test.json',bucket,'deepar/bikerental/data/test/test.json')

In [8]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [9]:
# We no longer have to maintain a mapping of container images by region
# Simply use the convenience method provided by sagemaker
# https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html
from sagemaker.amazon.amazon_estimator import get_image_uri
image_name = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')

In [10]:
image_name

'522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:1'

In [11]:
freq='H' # Timeseries consists Hourly Data and we need to predict hourly rental count

# how far in the future predictions can be made
# 12 days worth of hourly forecast 
prediction_length = 288 

# aws recommends setting context same as prediction length as a starting point. 
# This controls how far in the past the network can see
context_length = 288

In [12]:
# Check Free Tier (if you are still under free-tier)
# At this time, m4.xlarge is offered as part of 2 months free tier
# https://aws.amazon.com/sagemaker/pricing/
# If you are outside of free-tier, you can also use ml.m5.xlarge  (newer generation instance)
# In this example, I am using ml.m5.xlarge for training
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    base_job_name=base_job_name,
    output_path="s3://" + s3_output_path
)

In [13]:
freq, context_length, prediction_length

('H', 288, 288)

In [14]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "cardinality" : "auto" if with_categories else ''
}

In [15]:
hyperparameters

{'time_freq': 'H',
 'epochs': '400',
 'early_stopping_patience': '40',
 'mini_batch_size': '64',
 'learning_rate': '5E-4',
 'context_length': '288',
 'prediction_length': '288',
 'cardinality': ''}

In [16]:
estimator.set_hyperparameters(**hyperparameters)

In [17]:
# Here, we are simply referring to train path and test path
# You can have multiple files in each path
# SageMaker will use all the files
data_channels = {
    "train": "s3://{}/train/".format(s3_data_path),
    "test": "s3://{}/test/".format(s3_data_path)
}

In [18]:
data_channels

{'train': 's3://chandra-ml-sagemaker/deepar/bikerental/data/train/',
 'test': 's3://chandra-ml-sagemaker/deepar/bikerental/data/test/'}

In [19]:
# This step takes around 35 minutes to train the model with m4.xlarge instance
estimator.fit(inputs=data_channels)

2020-04-23 22:12:59 Starting - Starting the training job...
2020-04-23 22:13:01 Starting - Launching requested ML instances......
2020-04-23 22:14:05 Starting - Preparing the instances for training......
2020-04-23 22:15:05 Downloading - Downloading input data...
2020-04-23 22:15:52 Training - Training image download completed. Training in progress..Arguments: train
[04/23/2020 22:15:55 INFO 139961471010624] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience':

[04/23/2020 22:17:00 INFO 139961471010624] Epoch[2] Batch[0] avg_epoch_loss=3.544548
[04/23/2020 22:17:00 INFO 139961471010624] #quality_metric: host=algo-1, epoch=2, batch=0 train loss <loss>=3.54454803467
[04/23/2020 22:17:05 INFO 139961471010624] Epoch[2] Batch[5] avg_epoch_loss=3.488328
[04/23/2020 22:17:05 INFO 139961471010624] #quality_metric: host=algo-1, epoch=2, batch=5 train loss <loss>=3.48832829793
[04/23/2020 22:17:05 INFO 139961471010624] Epoch[2] Batch [5]#011Speed: 60.72 samples/sec#011loss=3.488328
[04/23/2020 22:17:09 INFO 139961471010624] processed a total of 636 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 14779.155015945435, "sum": 14779.155015945435, "min": 14779.155015945435}}, "EndTime": 1587680229.646922, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587680214.867705}

[04/23/2020 22:17:09 INFO 139961471010624] #throughput_metric: host=algo-1, train throughput=43.0332925259 records/secon

[04/23/2020 22:18:18 INFO 139961471010624] Epoch[7] Batch[0] avg_epoch_loss=3.071953
[04/23/2020 22:18:18 INFO 139961471010624] #quality_metric: host=algo-1, epoch=7, batch=0 train loss <loss>=3.07195281982
[04/23/2020 22:18:23 INFO 139961471010624] Epoch[7] Batch[5] avg_epoch_loss=3.048494
[04/23/2020 22:18:23 INFO 139961471010624] #quality_metric: host=algo-1, epoch=7, batch=5 train loss <loss>=3.0484940211
[04/23/2020 22:18:23 INFO 139961471010624] Epoch[7] Batch [5]#011Speed: 61.60 samples/sec#011loss=3.048494
[04/23/2020 22:18:27 INFO 139961471010624] processed a total of 604 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 14693.672895431519, "sum": 14693.672895431519, "min": 14693.672895431519}}, "EndTime": 1587680307.504144, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587680292.810413}

[04/23/2020 22:18:27 INFO 139961471010624] #throughput_metric: host=algo-1, train throughput=41.1058619542 records/second

[04/23/2020 22:19:38 INFO 139961471010624] Epoch[12] Batch[5] avg_epoch_loss=2.906964
[04/23/2020 22:19:38 INFO 139961471010624] #quality_metric: host=algo-1, epoch=12, batch=5 train loss <loss>=2.90696430206
[04/23/2020 22:19:38 INFO 139961471010624] Epoch[12] Batch [5]#011Speed: 60.95 samples/sec#011loss=2.906964
[04/23/2020 22:19:43 INFO 139961471010624] processed a total of 625 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 14785.629987716675, "sum": 14785.629987716675, "min": 14785.629987716675}}, "EndTime": 1587680383.122089, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587680368.336399}

[04/23/2020 22:19:43 INFO 139961471010624] #throughput_metric: host=algo-1, train throughput=42.2703382143 records/second
[04/23/2020 22:19:43 INFO 139961471010624] #progress_metric: host=algo-1, completed 3 % of epochs
[04/23/2020 22:19:43 INFO 139961471010624] #quality_metric: host=algo-1, epoch=12, train loss <loss>=2.8

[04/23/2020 22:20:53 INFO 139961471010624] Epoch[17] Batch[5] avg_epoch_loss=2.853821
[04/23/2020 22:20:53 INFO 139961471010624] #quality_metric: host=algo-1, epoch=17, batch=5 train loss <loss>=2.85382080078
[04/23/2020 22:20:53 INFO 139961471010624] Epoch[17] Batch [5]#011Speed: 61.79 samples/sec#011loss=2.853821
[04/23/2020 22:20:57 INFO 139961471010624] processed a total of 616 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 14636.117935180664, "sum": 14636.117935180664, "min": 14636.117935180664}}, "EndTime": 1587680457.29685, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587680442.660674}

[04/23/2020 22:20:57 INFO 139961471010624] #throughput_metric: host=algo-1, train throughput=42.0873894974 records/second
[04/23/2020 22:20:57 INFO 139961471010624] #progress_metric: host=algo-1, completed 4 % of epochs
[04/23/2020 22:20:57 INFO 139961471010624] #quality_metric: host=algo-1, epoch=17, train loss <loss>=2.84

...

[04/23/2020 22:47:28 INFO 139961471010624] Epoch[122] Batch[0] avg_epoch_loss=2.497370
[04/23/2020 22:47:28 INFO 139961471010624] #quality_metric: host=algo-1, epoch=122, batch=0 train loss <loss>=2.49737024307
[04/23/2020 22:47:33 INFO 139961471010624] Epoch[122] Batch[5] avg_epoch_loss=2.482946
[04/23/2020 22:47:33 INFO 139961471010624] #quality_metric: host=algo-1, epoch=122, batch=5 train loss <loss>=2.48294568062
[04/23/2020 22:47:33 INFO 139961471010624] Epoch[122] Batch [5]#011Speed: 61.48 samples/sec#011loss=2.482946
[04/23/2020 22:47:37 INFO 139961471010624] processed a total of 626 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 14689.244985580444, "sum": 14689.244985580444, "min": 14689.244985580444}}, "EndTime": 1587682057.578372, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587682042.888689}

[04/23/2020 22:47:37 INFO 139961471010624] #throughput_metric: host=algo-1, train throughput=42.6158019134 rec

[04/23/2020 22:49:01 INFO 139961471010624] Epoch[128] Batch[5] avg_epoch_loss=2.487455
[04/23/2020 22:49:01 INFO 139961471010624] #quality_metric: host=algo-1, epoch=128, batch=5 train loss <loss>=2.48745524883
[04/23/2020 22:49:01 INFO 139961471010624] Epoch[128] Batch [5]#011Speed: 61.94 samples/sec#011loss=2.487455
[04/23/2020 22:49:05 INFO 139961471010624] processed a total of 622 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 14582.728147506714, "sum": 14582.728147506714, "min": 14582.728147506714}}, "EndTime": 1587682145.665682, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587682131.082578}

[04/23/2020 22:49:05 INFO 139961471010624] #throughput_metric: host=algo-1, train throughput=42.6529294366 records/second
[04/23/2020 22:49:05 INFO 139961471010624] #progress_metric: host=algo-1, completed 32 % of epochs
[04/23/2020 22:49:05 INFO 139961471010624] #quality_metric: host=algo-1, epoch=128, train loss <loss

[04/23/2020 22:50:25 INFO 139961471010624] Epoch[134] Batch[0] avg_epoch_loss=2.514423
[04/23/2020 22:50:25 INFO 139961471010624] #quality_metric: host=algo-1, epoch=134, batch=0 train loss <loss>=2.51442265511
[04/23/2020 22:50:30 INFO 139961471010624] Epoch[134] Batch[5] avg_epoch_loss=2.470279
[04/23/2020 22:50:30 INFO 139961471010624] #quality_metric: host=algo-1, epoch=134, batch=5 train loss <loss>=2.47027870019
[04/23/2020 22:50:30 INFO 139961471010624] Epoch[134] Batch [5]#011Speed: 62.27 samples/sec#011loss=2.470279
[04/23/2020 22:50:34 INFO 139961471010624] processed a total of 595 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 14516.695022583008, "sum": 14516.695022583008, "min": 14516.695022583008}}, "EndTime": 1587682234.761521, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587682220.244391}

[04/23/2020 22:50:34 INFO 139961471010624] #throughput_metric: host=algo-1, train throughput=40.9869770837 rec

[04/23/2020 22:51:55 INFO 139961471010624] Epoch[140] Batch[0] avg_epoch_loss=2.538957
[04/23/2020 22:51:55 INFO 139961471010624] #quality_metric: host=algo-1, epoch=140, batch=0 train loss <loss>=2.53895688057
[04/23/2020 22:52:00 INFO 139961471010624] Epoch[140] Batch[5] avg_epoch_loss=2.526720
[04/23/2020 22:52:00 INFO 139961471010624] #quality_metric: host=algo-1, epoch=140, batch=5 train loss <loss>=2.52672028542
[04/23/2020 22:52:00 INFO 139961471010624] Epoch[140] Batch [5]#011Speed: 62.00 samples/sec#011loss=2.526720
[04/23/2020 22:52:05 INFO 139961471010624] Epoch[140] Batch[10] avg_epoch_loss=2.544873
[04/23/2020 22:52:05 INFO 139961471010624] #quality_metric: host=algo-1, epoch=140, batch=10 train loss <loss>=2.56665525436
[04/23/2020 22:52:05 INFO 139961471010624] Epoch[140] Batch [10]#011Speed: 61.45 samples/sec#011loss=2.566655
[04/23/2020 22:52:05 INFO 139961471010624] processed a total of 644 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 15708.613157


2020-04-23 22:53:45 Uploading - Uploading generated training model#metrics {"Metrics": {"model.score.time": {"count": 1, "max": 16841.02702140808, "sum": 16841.02702140808, "min": 16841.02702140808}}, "EndTime": 1587682420.673546, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587682403.832611}

[04/23/2020 22:53:40 INFO 139961471010624] #test_score (algo-1, RMSE): 40.1434449833
[04/23/2020 22:53:40 INFO 139961471010624] #test_score (algo-1, mean_absolute_QuantileLoss): 16355.638567985097
[04/23/2020 22:53:40 INFO 139961471010624] #test_score (algo-1, mean_wQuantileLoss): 0.13587803080489402
[04/23/2020 22:53:40 INFO 139961471010624] #test_score (algo-1, wQuantileLoss[0.1]): 0.07697326526651156
[04/23/2020 22:53:40 INFO 139961471010624] #test_score (algo-1, wQuantileLoss[0.2]): 0.12001783375002434
[04/23/2020 22:53:40 INFO 139961471010624] #test_score (algo-1, wQuantileLoss[0.3]): 0.14707989335099733
[04/23/2020 22:53:40 INFO 139961

In [20]:
job_name = estimator.latest_training_job.name

In [21]:
# Hard code name for now as we stopped the notebook.  
# If you do this in a single sitting, you don't need to hard code
# job_name = 'deepar-biketrain-with-categories-2018-12-21-04-05-44-478'

In [22]:
print ('job name: {0}'.format(job_name))

job name: deepar-biketrain-no-categories-2020-04-23-22-12-59-148


In [23]:
# Create an endpoint for real-time predictions
endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    deployment_image=image_name,
    role=role
)

-----------------!

In [24]:
print ('endpoint name: {0}'.format(endpoint_name))

endpoint name: deepar-biketrain-no-categories-2020-04-23-22-12-59-148


In [25]:
# Don't forget to terminate the end point after completing the demo
# Otherwise, you account will accumulate hourly charges

# you can delete from sagemaker management console or through command line or throught code

# sagemaker_session.delete_endpoint(endpoint_name)